In [3]:
# https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-create-workspace-with-python

import azureml.core
print(azureml.core.VERSION)

0.1.74


In [4]:
import os
from azureml.core import Workspace

rg = "MLDeploy-docs"
ws_name = "mldeploydocs"
loc = "eastus2"
sub_key = os.getenv("AZURE_SUBSCRIPTION")

try: # Try to get, if it fails create a new one
    ws = Workspace.get(ws_name, subscription_id=sub_key)
except:
    ws = Workspace.create(name=ws_name,
                          subscription_id=sub_key,
                          resource_group=rg,
                          create_resource_group=False,
                          location=loc)
    # Save workspace details in aml_config/config.json
    ws.write_config() # use ws = Workspace.from_config() in the future 
ws.get_details()

{'id': '/subscriptions/b7442be7-20fd-472a-ad3b-407b9510af5a/resourceGroups/MLDeploy-docs/providers/Microsoft.MachineLearningServices/workspaces/mldeploydocs',
 'name': 'mldeploydocs',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'location': 'eastus2',
 'identity': {'principal_id': '7c3d75df-b787-461a-99f6-1f38563e9c3c',
  'tenant_id': '72f988bf-86f1-41af-91ab-2d7cd011db47',
  'type': 'SystemAssigned'}}

In [5]:
from azureml.core import Experiment
from keras.datasets import mnist
from sklearn.linear_model import LogisticRegression
import numpy as np

# Sample experiment and logging
exp = Experiment(workspace=ws, name='sklearn-mnist')

run = exp.start_logging()
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Flatten image data, current shape is (nsamples, 28, 28), we want (nsamples, 28*28,)
X_train = X_train.reshape((len(X_train), 28*28))
X_test = X_test.reshape((len(X_test), 28*28))

# # Then we normalize our channel data.
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)
acc = np.average(clf.predict(X_test) == y_test)
print(acc)

c:\users\jaearle\appdata\local\continuum\miniconda3\envs\myenv\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\jaearle\appdata\local\continuum\miniconda3\envs\myenv\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.9202


In [6]:
run.log("seed", 42)
run.log("accuracy", acc)
run.complete()
run.get_portal_url()

'https://mlworkspace.azure.ai/portal/subscriptions/b7442be7-20fd-472a-ad3b-407b9510af5a/resourceGroups/MLDeploy-docs/providers/Microsoft.MachineLearningServices/workspaces/mldeploydocs/experiments/sklearn-mnist/runs/46750002-dd3c-4309-875f-098a4f08ef79'